# Test des données récupérées par API

[Encodage-pourcent](https://fr.wikipedia.org/wiki/Encodage-pourcent)

Ce fichier prend en compte les requêtes pour l'appel API

À travailler : https://opendata.paris.fr/explore/dataset/logement-encadrement-des-loyers/api/?disjunctive.annee&disjunctive.id_zone&disjunctive.nom_quartier&disjunctive.piece&disjunctive.epoque&disjunctive.meuble_txt

Objet du traitement : encadrement du loyer à Paris

Date : 04-01-2024 <br>
Éditeur : <b>Laurent REYNAUD</b>

In [1]:
import requests
import json
import polars as pl

### URL API :

In [3]:
url_api = "/api/explore/v2.1/catalog/datasets/logement-encadrement-des-loyers/records?limit=20"

### Variables à saisir avec les requêtes SQL WHERE / LIMIT... ci-après :

In [75]:
year = ['2019', '2020', '2021', '2022', '2023'] # donnée "annee"
# geo = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] # donnée 'id_zone'
df_districts = pl.read_csv('data/districts.csv') # donnée 'nom_quartier'
list_districts = df_districts.to_numpy().tolist() # conversion df en liste
districts = [district for list_district in list_districts
             for district in list_district]
rooms = ['1', '2', '3', '4'] # donnée 'piece'
period = ["Avant 1946", "1946-1970", "1971-1990", "Apres 1990"] # donnée 'epoque'
furnished = ["meublé", "non meublé"] # donnée 'meuble_txt'

### Requête SQL avec SELECT :

annee, nom_quartier, piece, epoque, meuble_txt, ref, min, max

### Requête SQL avec WHERE :

annee=2022 and nom_quartier="Madeleine" and piece=1 and epoque="Apres 1990" and meuble_txt="non meublé"

### Requête SQL avec GROUP_BY :

### Requête SQL avec ORDER BY :

ref

### Requête SQL avec LIMIT :

100

### Intervention sur python

In [92]:
# Accès à l'URL avec le requêtage appliqué : 
# url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/logement-encadrement-des-loyers/records?select=annee%2C%20id_zone%2C%20nom_quartier%2C%20piece%2C%20epoque%2C%20meuble_txt%2C%20ref%2C%20min%2C%20max&where=annee%3D2022%20and%20nom_quartier%3D%22Madeleine%22%20and%20piece%3D1%20and%20epoque%3D%22Apres%201990%22%20and%20meuble_txt%3D%22non%20meubl%C3%A9%22&order_by=ref&limit=100"
url = f"https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/logement-encadrement-des-loyers/records?select=annee%2C%20nom_quartier%2C%20piece%2C%20epoque%2C%20meuble_txt%2C%20ref%2C%20min%2C%20max&where=annee%3D{year[3]}%20and%20nom_quartier%3D%22{districts[5]}%22%20and%20piece%3D{rooms[1]}%20and%20epoque%3D%22{period[2]}%22%20and%20meuble_txt%3D%22{furnished[1]}%22&order_by=ref&limit=100"

In [93]:
# Requête url
response = requests.get(url)

In [94]:
# Code statut html
response.status_code

200

In [95]:
# Récupération du contenu au format json
data = json.loads(response.content)

In [96]:
# Affichage des données
df = pl.DataFrame(data['results'])


In [97]:
df

annee,id_zone,nom_quartier,piece,epoque,meuble_txt,ref,min,max
str,i64,str,i64,str,str,f64,f64,f64
"""2022""",10,"""Batignolles""",2,"""1971-1990""","""non meublé""",23.1,16.17,27.72
